In [1]:
import pandas as pd
from datetime import datetime, timedelta
from glob import glob
import numpy as np
from random import randint, choice



import statsmodels.api as sm
import matplotlib.pyplot as plt


import dash
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objs as go
from dash import Input, Output, dcc, html



CSV_TOPOLOGY = 'internet_headers_v126.csv'

DIR_DB = '/home/valdo/raw_data/Floki/Internet_BW_ports/*'
DIR_REGIONES = 'regiones_localidades.csv'
REPORT = 'HFC'.lower()

In [2]:
_index_cols = ['Instancia 0','Instancia 1','Instancia 2','Extremo A','Pta A', 'Capacidad']

TOPOLOGY_FILE = pd.read_csv(CSV_TOPOLOGY, sep=';')
TOPOLOGY_FILE['Pta A'] = TOPOLOGY_FILE['Pta A'].str.replace("'","")
TOPOLOGY_FILE[_index_cols[0:-1]] = TOPOLOGY_FILE[_index_cols[0:-1]].applymap(lambda x: x.lower())
TOPOLOGY_FILE[_index_cols[-1]] = pd.to_numeric(TOPOLOGY_FILE[_index_cols[-1]])
TOPOLOGY_FILE = TOPOLOGY_FILE.set_index(_index_cols)


### Retrieving DB about Regiones Chile

In [3]:
REGIONES_CSV = pd.read_csv(DIR_REGIONES, sep=';')
REGIONES = {v.Reg:{"Region":v.Region,"Localidad":v.Localidad} for k,v in REGIONES_CSV.iterrows()}


### Generating DB of Colors

In [4]:
COLORS_DB = pd.read_csv('colors.csv',sep=',', header=None)
COLORS_DB.columns = ['name_code','name','hex','r','g','b']


### Retrieving Information about BandWidth

In [5]:
def check_outliners(row:pd.Series):
    q25 = row.quantile(0.25)
    q75 = row.quantile(0.75)

    _limit_up = 1.5*(q75 - q25) + q75
    _limit_dw = -1.5*(q75 - q25) + q25

       
    row.loc[row > _limit_up] = row.mean()
    row.loc[row < _limit_dw] = row.mean()
    
    return row.fillna(row.mean())
    
def read_file_from_week(files: pd.DataFrame, week = 0):
    _week_data = pd.DataFrame()
    
    if week > 0:
        _files = files[files['week'] == week]
    else:
        print("No week defined:  Reading all")
        _files = files
    
    for k,file in _files.iterrows():
        print("Reading file: %s"%file['path'])
        _raw_data = pd.read_csv(file['path'], sep=';')        
        _raw_data.index = pd.MultiIndex.from_tuples( _raw_data.iloc[:,0].str.lower().str.split('&101&').apply(lambda x: tuple([x.replace("'","") for x in x])) )
        _raw_data = _raw_data.iloc[:,1:]
        _raw_data.columns = [pd.to_datetime(x) for x in _raw_data.columns]
        _week_data = pd.concat([_week_data,_raw_data], axis=1)
    
    return _week_data

In [6]:
## RETRIEVING FILES of RAWDATA ########################################
FILES = pd.DataFrame([name for name in glob(DIR_DB)], columns=['path'])
FILES['date'] = pd.to_datetime(FILES['path'].str.extract(r'(.*)internet_headers_(\d+)')[1], format='%Y%m%d') 
FILES['date'] = FILES['date'].apply(lambda x: x - timedelta(days=1))
FILES['week'] = FILES['date'].apply(lambda x: x.isocalendar()[1])
FILES = FILES.sort_values(by='date')
FILES = FILES.reset_index(drop=True)


## READING FILES of RAWDATA ########################################
WEEK_DATE = read_file_from_week(FILES)
WEEK_DATE = WEEK_DATE.apply(lambda x:  x.fillna(x.mean()))


RANGE_DATE_TITLE = WEEK_DATE.columns.min().strftime("%b W%W") +" to " + WEEK_DATE.columns.max().strftime("%b W%W")
RANGE_DATE_TITLE_MIN = WEEK_DATE.columns.min()
RANGE_DATE_TITLE_MAX = WEEK_DATE.columns.max()

No week defined:  Reading all
Reading file: /home/valdo/raw_data/Floki/Internet_BW_ports/internet_headers_20220312.csv
Reading file: /home/valdo/raw_data/Floki/Internet_BW_ports/internet_headers_20220313.csv
Reading file: /home/valdo/raw_data/Floki/Internet_BW_ports/internet_headers_20220314.csv
Reading file: /home/valdo/raw_data/Floki/Internet_BW_ports/internet_headers_20220315.csv
Reading file: /home/valdo/raw_data/Floki/Internet_BW_ports/internet_headers_20220316.csv
Reading file: /home/valdo/raw_data/Floki/Internet_BW_ports/internet_headers_20220317.csv
Reading file: /home/valdo/raw_data/Floki/Internet_BW_ports/internet_headers_20220318.csv
Reading file: /home/valdo/raw_data/Floki/Internet_BW_ports/internet_headers_20220319.csv
Reading file: /home/valdo/raw_data/Floki/Internet_BW_ports/internet_headers_20220320.csv
Reading file: /home/valdo/raw_data/Floki/Internet_BW_ports/internet_headers_20220321.csv
Reading file: /home/valdo/raw_data/Floki/Internet_BW_ports/internet_headers_2022

### Ocupación Total

In [7]:
HFC_DATA = WEEK_DATE.copy()
HFC_DATA = HFC_DATA.loc[pd.IndexSlice['internet',['hfc','ftth'],:,:,:,:,:]].loc['internet']
HFC_DATA = HFC_DATA[HFC_DATA.mean(axis=1) > 0]
HFC_DATA = HFC_DATA.T.resample('D').max().T
HFC_DATA = HFC_DATA.groupby(level=[0,1,2,3,4]).max()
HFC_DATA = HFC_DATA.sum().apply(lambda x: np.round(x/1000,1)).to_frame().rename(columns={0:'BW [Mbps]'})
HFC_DATA = HFC_DATA.apply(check_outliners)
HFC_DATA = HFC_DATA.apply(lambda x: x.ewm(com=0.6, adjust=True).mean())
HFC_DATA['Trend'] = sm.tsa.seasonal_decompose(HFC_DATA[HFC_DATA.columns[0]], model='add').trend

fig_total_1 = px.line(HFC_DATA, y=[HFC_DATA.columns[0], HFC_DATA.columns[1]],  title="Utilizacion de Ancho de Banda Total: "+RANGE_DATE_TITLE)


fig_total_1.update_layout(
    margin=dict(l=20, r=20, t=30, b=20),
    paper_bgcolor="LightSteelBlue",
    height=500,
    yaxis_title="Gbps",
    xaxis_title="Tiempo",
)


None

### Grafico de Totales por Region 

In [8]:
HFC_DATA = WEEK_DATE.copy()
HFC_DATA = HFC_DATA.loc[pd.IndexSlice['internet',['hfc','ftth'],:,:,:,:,:]].loc['internet']
HFC_DATA = HFC_DATA[HFC_DATA.mean(axis=1) > 0]
HFC_DATA = HFC_DATA.T.resample('D').max().T
HFC_DATA = HFC_DATA.groupby(level=[0,1,2,3,4]).max()
HFC_DATA = HFC_DATA.apply(check_outliners, axis=1)
HFC_DATA = HFC_DATA.apply(lambda x: x.ewm(com=0.6, adjust=True).mean(), axis=1)/1000
HFC_DATA = HFC_DATA.groupby(level=[1]).sum()
HFC_DATA = HFC_DATA.stack().to_frame()
HFC_DATA.reset_index(inplace=True)
HFC_DATA.columns = ['Localidad','Date','Bw']
HFC_DATA['Week'] = HFC_DATA['Date'].apply(lambda x: x.isocalendar()[1])
HFC_DATA['Localidad'] = HFC_DATA['Localidad'].apply(lambda x: REGIONES[x]['Localidad'])


fig_cmts_2 = px.line(HFC_DATA,
                     x='Date', 
                     y='Bw', 
                     color='Localidad', 
                     symbol="Localidad",
                     color_discrete_sequence=[choice(COLORS_DB['hex']) for x in range(len(HFC_DATA)) ],
                     title="Historial de Uso "+RANGE_DATE_TITLE)


fig_cmts_2.update_layout(
    margin=dict(l=20, r=20, t=30, b=20),
    paper_bgcolor="LightSteelBlue",
    height=800,
    yaxis_title="Gbps",
    xaxis_title="Tiempo",
)

None

In [9]:
###TABLA CMTS
HFC_DATA = WEEK_DATE.copy()
HFC_DATA = HFC_DATA.loc[pd.IndexSlice['internet',['hfc','ftth'],:,:,:,:,:]].loc['internet']
HFC_DATA = HFC_DATA[HFC_DATA.mean(axis=1) > 0]
HFC_DATA = HFC_DATA.T.resample('D').max().T
HFC_DATA = HFC_DATA.groupby(level=[0,1,2,3,4]).max()
HFC_DATA = HFC_DATA.apply(check_outliners, axis=1)
HFC_DATA = HFC_DATA.apply(lambda x: x.ewm(com=0.6, adjust=True).mean(), axis=1)/1000
HFC_DATA = HFC_DATA.groupby(level=[1]).sum()
HFC_DATA = HFC_DATA.T.resample('W').max().T
HFC_DATA = HFC_DATA[HFC_DATA.columns[-7:]]
HFC_DATA.columns = ['W'+str(x.isocalendar().week) for x in HFC_DATA.columns]
HFC_DATA = HFC_DATA.applymap(lambda x: np.round(x,1))
HFC_DATA.reset_index(inplace=True)
#HFC_DATA['index'] =  HFC_DATA['index'].str.upper()
HFC_DATA['Localidad'] = [REGIONES[x.lower()]['Localidad'] for x in HFC_DATA['index']]
HFC_DATA  = HFC_DATA.drop('index', axis=1)
HFC_DATA = HFC_DATA[ ['Localidad'] + [x for x in HFC_DATA.columns if x != 'Localidad']]


fig_table_1 = go.Figure(data=[go.Table(
    header=dict(values=list(HFC_DATA.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=HFC_DATA.T.values,
               fill_color='lavender',
               align='left'))
])

fig_table_1.update_layout(
    margin=dict(l=20, r=20, t=30, b=20),
    paper_bgcolor="LightSteelBlue",
    height=600,
    title="Tabla Historica de Demanda por usuario HFC  [Gbps]"
)

None

### Utilizacion de CMTS

In [10]:
###### DEV
HFC_DATA = WEEK_DATE.copy()
HFC_DATA = HFC_DATA.loc[pd.IndexSlice['internet',['hfc','ftth'],:,:,:,:,:]].loc['internet']
HFC_DATA = HFC_DATA[HFC_DATA.mean(axis=1) > 0]
HFC_DATA = HFC_DATA.T.resample('D').max().T
HFC_DATA = HFC_DATA.groupby(level=[0,1,2,3,4]).max()
HFC_DATA = HFC_DATA.apply(check_outliners, axis=1)
HFC_DATA = HFC_DATA.apply(lambda x: x.ewm(com=0.6, adjust=True).mean(), axis=1)
HFC_DATA = HFC_DATA.max(axis=1).to_frame().rename(columns={0:'BW'})
HFC_DATA['Capacity'] = pd.to_numeric(HFC_DATA.index.get_level_values(-1))
HFC_DATA = HFC_DATA.droplevel([-1])
HFC_DATA = HFC_DATA.groupby(level=[0,1]).sum().applymap(lambda x: int(np.round(x/1000,0)))
HFC_DATA['Used'] = HFC_DATA['BW']
HFC_DATA['Free'] = HFC_DATA['Capacity'] - HFC_DATA['BW']
HFC_DATA.reset_index(inplace=True)
HFC_DATA = HFC_DATA.rename(columns={'level_1':'CMTS', 'level_0':'Medio'})
HFC_DATA['Localidad'] = HFC_DATA['CMTS'].apply(lambda x: REGIONES[x]['Localidad'])
HFC_DATA['Medio'] = HFC_DATA['Medio'].apply(lambda x: 'FTTH' if x == 'ftth' else 'HFC')
HFC_DATA['Utilization%'] = HFC_DATA.apply(lambda x: 100*(x['Used']/x['Capacity']), axis=1)

fig_cmts_3 = px.bar(
    HFC_DATA, 
    x= "Localidad", 
    y= "Capacity", 
    color='Utilization%',
    color_continuous_scale='icefire',
    range_color=(0,100),
    title="Utilizacion por CMTS: "+RANGE_DATE_TITLE,
    barmode="group",
    facet_col="Medio"
)

fig_cmts_3.update_layout(
    margin=dict(l=20, r=20, t=30, b=20),
    paper_bgcolor="LightSteelBlue",
    height=500,
    yaxis_title="Gbps",
    xaxis_title="Localidad",
)

None


### Diagrama de MPLS

In [11]:
### DIAGRAMA   P - PE
DEST_NODES = TOPOLOGY_FILE.copy()
DEST_NODES = DEST_NODES.loc[pd.IndexSlice['residencial','mpls','mpls',:,:,:,:]]
DEST_NODES = DEST_NODES[['Extremo B']]
DEST_NODES['Extremo B'] = DEST_NODES['Extremo B'].str.lower()
DEST_NODES = DEST_NODES.droplevel('Capacidad')
DEST_NODES.reset_index(inplace=True)


HFC_DATA = WEEK_DATE.copy()
HFC_DATA = HFC_DATA.loc[pd.IndexSlice['residencial','mpls','mpls',:,:,:,:]]
HFC_DATA = HFC_DATA[HFC_DATA.mean(axis=1) > 0]
HFC_DATA = HFC_DATA.T.resample('D').max().T
HFC_DATA = HFC_DATA.groupby(level=[0,1,2]).max()
HFC_DATA = HFC_DATA.apply(check_outliners, axis=1)
HFC_DATA = HFC_DATA.apply(lambda x: x.ewm(com=0.6, adjust=True).mean(), axis=1)
HFC_DATA = HFC_DATA.max(axis=1).to_frame().rename(columns={0:'BW'})
HFC_DATA.reset_index(inplace=True)
HFC_DATA.columns = ['Extremo A', 'Pta A','Capacidad','BW']
HFC_DATA = HFC_DATA.merge(DEST_NODES, on=['Extremo A','Pta A'], how='left')
HFC_DATA['Capacidad'] = HFC_DATA['Capacidad'].apply(lambda x: pd.to_numeric(x))
HFC_DATA.loc[:,['Extremo A','Extremo B']] = HFC_DATA.loc[:,['Extremo A','Extremo B']].apply(lambda x: x.str.upper())
HFC_DATA['A'] = HFC_DATA['Extremo A'].str.replace('\d','',  regex=True)
HFC_DATA['B'] = HFC_DATA['Extremo B'].str.replace('\d','',  regex=True)


HFC_DATA = HFC_DATA.groupby(['A','B']).sum()
HFC_DATA.reset_index(inplace=True)
HFC_DATA['Label'] = HFC_DATA.apply(lambda x: "%0.0f%% - %d Gbps"%( 100*(x['BW']/x['Capacidad']), x['Capacidad']/1000), axis=1)


_list_nodes = list(HFC_DATA['A'].unique()) +  list( HFC_DATA['B'].unique())
_list_nodes_map = {v:i for i,v in enumerate(_list_nodes)}
_list_nodes_map



fig_sankey_1 = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = " Gbps",
    
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  _list_nodes,
    ),
    
    link = dict(
      source =  [_list_nodes_map[x] for x in HFC_DATA['A']],
      target =  [_list_nodes_map[x] for x in HFC_DATA['B']], 
      value= list(HFC_DATA['BW']/1000),
      label= list(HFC_DATA['Label'])

))])


fig_sankey_1.update_layout(
    margin=dict(l=20, r=20, t=30, b=20),
    paper_bgcolor="LightSteelBlue",
    height=900,
    title='Anchos de Banda'
)

None

In [12]:
### CACHES - CONTENIDO
HFC_DATA = WEEK_DATE.copy()
HFC_DATA = HFC_DATA.loc['internet','cache-contenido']
HFC_DATA = HFC_DATA[HFC_DATA.mean(axis=1) > 0]
HFC_DATA = HFC_DATA.T.resample('D').max().T
HFC_DATA = HFC_DATA.groupby(level=[0,1,2,3]).max()
HFC_DATA = HFC_DATA.apply(check_outliners, axis=1)
HFC_DATA = HFC_DATA.apply(lambda x: x.ewm(com=0.6, adjust=True).mean(), axis=1)
HFC_DATA = HFC_DATA.max(axis=1).to_frame().rename(columns={0:'BW'})
HFC_DATA.reset_index(inplace=True)
HFC_DATA.columns = ['Instancia 1','Extremo A', 'Pta A','Capacidad','BW']
HFC_DATA = HFC_DATA[ HFC_DATA['Extremo A'].str.match(r'sw-cache-([a-z]+)')]
HFC_DATA['B'] = HFC_DATA['Extremo A'].str.extract(r'sw-cache-([a-z]+)')[0]
HFC_DATA['A'] = HFC_DATA['Instancia 1']
HFC_DATA['Capacidad'] = pd.to_numeric(HFC_DATA['Capacidad'])
HFC_DATA = HFC_DATA[ HFC_DATA['Capacidad'] > 0 ]
HFC_DATA['B'] = HFC_DATA['B'].str.upper()#.apply(lambda x: x+"-HPE")
HFC_DATA['A'] = HFC_DATA['A'].str.upper()
HFC_DATA = HFC_DATA[['A','B','BW','Capacidad']]

HFC_DATA = HFC_DATA.groupby(['A','B']).sum()
HFC_DATA.reset_index(inplace=True)
HFC_DATA['Label'] = HFC_DATA.apply(lambda x: "%0.0f%% - %d Gbps"%( 100*(x['BW']/x['Capacidad']), x['Capacidad']/1000), axis=1)


_list_nodes = list(HFC_DATA['A'].unique()) +  list( HFC_DATA['B'].unique())
_list_nodes_map = {v:i for i,v in enumerate(_list_nodes)}

fig_sankey_2 = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = " Gbps",
    
    node = dict(
      pad = 15,
      thickness = 15,
      line = dict(color = "black", width = 0.5),
      label =  _list_nodes,
    ),
    
    link = dict(
      source =  [_list_nodes_map[x] for x in HFC_DATA['A']],
      target =  [_list_nodes_map[x] for x in HFC_DATA['B']], 
      value= list(HFC_DATA['BW']/1000),
      label= list(HFC_DATA['Label'])

))])


fig_sankey_2.update_layout(
    margin=dict(l=20, r=20, t=30, b=20),
    paper_bgcolor="LightSteelBlue",
    height=600,
    title='Utilización de CACHES'
)


HFC_DATA_PIE  = HFC_DATA.groupby(['A']).sum()
HFC_DATA_PIE = HFC_DATA_PIE[HFC_DATA_PIE['BW'] > 100]
HFC_DATA_PIE['FREE'] = HFC_DATA_PIE['Capacidad'] -  HFC_DATA_PIE['BW']
HFC_DATA_PIE = HFC_DATA_PIE/1000
HFC_DATA_PIE = HFC_DATA_PIE.astype(int)
HFC_DATA_PIE = HFC_DATA_PIE.reset_index(inplace=False)

#fig_pie_1 = px.pie(HFC_DATA_PIE, values='BW', names='A')
fig_pie_1 =go.Figure(go.Sunburst(
    labels=[x for x  in HFC_DATA_PIE['A']] + [x+"_USED" for x  in HFC_DATA_PIE['A']] + [x+"_FREE" for x  in HFC_DATA_PIE['A']],
    parents=["" for x  in HFC_DATA_PIE['A']] + [x for x  in HFC_DATA_PIE['A']] + [x for x  in HFC_DATA_PIE['A']],
    values=[x for x  in HFC_DATA_PIE['Capacidad']] + [x for x  in HFC_DATA_PIE['BW']] + [x for x  in HFC_DATA_PIE['FREE']],
    branchvalues="total",
    textinfo= 'label+percent entry',
    
))

fig_pie_1.update_layout(
    uniformtext_minsize=12, 
    uniformtext_mode='hide',
    margin=dict(l=20, r=20, t=30, b=20),
    paper_bgcolor="LightSteelBlue",
    height=400,
    title='')



_cols = ['Localidad','BW','Capacidad']

fig_table_2 = go.Figure(data=[go.Table(
    header=dict(values=list(_cols),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=HFC_DATA_PIE[[x for x in HFC_DATA_PIE.columns if x != 'FREE']].T.values,
               fill_color='lavender',
               align='left'))
])

fig_table_2.update_layout(
    margin=dict(l=10, r=10, t=30, b=1),
    paper_bgcolor="LightSteelBlue",
    height=200,
    title="Cache de Contenido"
)

None

/tmp/ipykernel_21449/1588781833.py:3: PerformanceWarning:

indexing past lexsort depth may impact performance.



## CREACION DE DASHBOARD

In [ ]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])


app.layout = dbc.Container(
    [
        html.H1("Informe de Capacidades de Residencial:   De "+RANGE_DATE_TITLE_MIN.strftime("%b %d")+" hasta "+RANGE_DATE_TITLE_MAX.strftime("%b %d")),
        
        html.Hr(),
        dcc.Markdown('''
        *Utilizacion Total de la Capacidad de Ancho del Servicio Residencial*
        '''),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_total_1))
        ]),
        
        html.Br(),
        html.Br(),
        dcc.Markdown('''
        *Historial de Utilización de ancho de Banda por cada uno de los CMTs*
        *Existe una caida importante de Utilización en la localidad de Concepción*
        '''),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_cmts_2))
        ]),
        
        
        html.Br(),
        html.Br(),
        dcc.Markdown('''
        *Utilización Total de La Capacidad de los CMTS*
        *La Unidad de Negocio en FTTH tiene una utilización cercano de 20%*
        *La Unidad de Negocio en HFC tiene una utilizaciòn cercano al 40%, y se toma en cuenta los casos de Los Angeles y Copiapo, con ocupaciones cercanos al 60%*
        '''),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_cmts_3))
        ]),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_table_1))
        ]),
        html.Br(),
        html.Br(),
        dcc.Markdown('''
        *Utilización de Capacidad y Ancho de Banda de enlaces de la MPLS*
        '''),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_sankey_1)),
        ]),
        dbc.Row([]),
        dbc.Row([
            dbc.Col(dcc.Graph(figure=fig_sankey_2)),
            dbc.Col([
                dcc.Graph(figure=fig_pie_1),
                dcc.Graph(figure=fig_table_2),
                ])
        ]),
        
        html.Br(),
        html.Br(),
    ]
)




if __name__ == "__main__":

    app.run_server(host='0.0.0.0',  port=8155)

Dash is running on http://0.0.0.0:8155/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8155/ (Press CTRL+C to quit)
172.19.0.1 - - [02/May/2022 15:58:24] "GET / HTTP/1.1" 200 -
172.19.0.1 - - [02/May/2022 15:58:25] "GET /_dash-dependencies HTTP/1.1" 200 -
172.19.0.1 - - [02/May/2022 15:58:25] "GET /_favicon.ico?v=2.3.1 HTTP/1.1" 200 -
172.19.0.1 - - [02/May/2022 15:58:25] "GET /_dash-layout HTTP/1.1" 200 -
172.19.0.1 - - [02/May/2022 15:58:25] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
172.19.0.1 - - [02/May/2022 15:58:25] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 200 -
172.19.0.1 - - [02/May/2022 15:58:25] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
172.19.0.1 - - [02/May/2022 15:58:26] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 200 -
172.19.0.1 - - [02/May/2022 16:51:57] "GET / HTTP/1.1" 200 -
172.19.0.1 - - [02/May/2022 16:51:58] "GET /_dash-dependencies HTTP/1.1" 200 -
172.19.0.1 - - [02/May/2022 16:51:58] "GET /_dash-layout HTTP/1.1" 200